In [1]:
# Esse codigo é um exemplo de como criar um ambiente virtual e instalar dependências em Python.
#pip install -r requirements.txt

## 1- Organizando os dados e bibliotecas 

In [2]:
# Bibliotecas necessárias
import pandas as pd
import cvxpy as cp


In [3]:


# Carregar a planilha
df = pd.read_excel('data_real.xlsx',sheet_name='Planilha2')
df.columns = df.columns.str.strip()

# 🔁 ORDENAR antes de numerar
df = df.sort_values('Banco').reset_index(drop=True)

# Gerar número incremental por banco
df['numero'] = df.groupby('Banco').cumcount() + 1
df['DMU'] = df['Banco'] + '_' + df['numero'].astype(str).str.zfill(2)

# Selecionar colunas no formato DEA
df_dea = df[[
    'DMU',
    'Despesa de Intermediação Financeira',
    'Despesa Administrativa e Operacionais',
    'Receita de Intermediação Financeira',
    'Receita com prestação de serviços',
    'Outras receitas operacionais'
]]


In [4]:
# Salvar em CSV
df_dea.to_csv('dados_dea.csv', index=False)

## 2- Função para rodar modelo DEA de minimização

In [5]:
# Carrega os dados do CSV
df = pd.read_csv("dados_dea.csv")

In [6]:
# Definir inputs e outputs
inputs = [
    "Despesa de Intermediação Financeira",
    "Despesa Administrativa e Operacionais"
]
outputs = [
    "Receita de Intermediação Financeira",
    "Receita com prestação de serviços",
    "Outras receitas operacionais"
]

In [7]:
# Corrigir: tornar os inputs positivos
df[inputs] = df[inputs].abs()

# (Opcional) Reduzir escala para facilitar o cálculo numérico
df[inputs + outputs] = df[inputs + outputs] / 1e6

In [8]:
# Extrair as matrizes para o modelo DEA
X = df[inputs].values
Y = df[outputs].values
n = X.shape[0]

In [9]:
# === FUNÇÃO GERAL PARA RODAR DEA ===
def run_dea_minimizacao(X, Y, orientation="input", rts="crs"):
    efficiencies = []
    for j in range(n):
        lambdas = cp.Variable(n, nonneg=True)
        theta = cp.Variable()

        constraints = []

        # Orientação a input
        constraints += [X.T @ lambdas <= theta * X[j]]
        constraints += [Y.T @ lambdas >= Y[j]]

        if rts == "crs":
            pass  # sem restrição de soma dos lambdas
        elif rts == "vrs":
            constraints += [cp.sum(lambdas) == 1]
            
        problem = cp.Problem(cp.Minimize(theta), constraints)
        problem.solve()

        value = float(theta.value) if theta.value is not None else None
        efficiencies.append(round(value, 4) if value is not None else None)
    return efficiencies

In [10]:
eff_crs = run_dea_minimizacao(X, Y, rts="crs")
eff_vrs = run_dea_minimizacao(X, Y, rts="vrs")

## 3- Função para rodar modelo DEA de maximização

In [11]:
def run_dea_output(X, Y, rts="crs"):
    """
    DEA output-oriented:
    - Maximiza o fator phi de expansão de outputs
    - Restrições de insumos fixos e outputs escalonados
    - rts: "crs" (sem restrição) ou "vrs" (convexidade)
    """
    n = X.shape[0]
    efficiencies = []

    for j in range(n):
        lambdas = cp.Variable(n, nonneg=True)
        phi     = cp.Variable(nonneg=True)

        constraints = []
        # 1) Insumos: combinação convexa não pode ultrapassar insumos atuais
        constraints += [X.T @ lambdas <= X[j]]
        # 2) Outputs: combinação deve ser pelo menos phi * outputs atuais
        constraints += [Y.T @ lambdas >= phi * Y[j]]

        # 3) Se VRS, impõe soma dos lambdas = 1
        if rts.lower() == "vrs":
            constraints += [cp.sum(lambdas) == 1]

        # Problema de maximização de phi
        problem = cp.Problem(cp.Maximize(phi), constraints)
        problem.solve()  # você pode especificar solver e tolerâncias aqui

        val = phi.value if phi.value is not None else None
        efficiencies.append(round(float(val), 4) if val is not None else None)

    return efficiencies

# Exemplo de uso:
eff_output_crs = run_dea_output(X, Y, rts="crs")
eff_output_vrs = run_dea_output(X, Y, rts="vrs")

## 4- Resultados do DEA

In [12]:
# === RESULTADOS ===
df_resultado = pd.DataFrame({
    "Banco": df["DMU"].str.extract(r"(^[A-Za-z]+)")[0],
    "DMU": df["DMU"],
    "Eff_CRS_MIN": eff_crs,
    "Eff_VRS_MIN": eff_vrs
})

# Incluindo no DataFrame de resultados:
df_resultado["Eff_CRS_MAX"] = eff_output_crs
df_resultado["Eff_VRS_MAX"] = eff_output_vrs



In [13]:
# Média das eficiências por banco a partir de df_resultado
df_media = df_resultado.groupby("Banco")[["Eff_CRS_MIN", "Eff_VRS_MIN", "Eff_CRS_MAX", "Eff_VRS_MAX"]].mean().round(4).reset_index()
df_media

,Banco,Eff_CRS_MIN,Eff_VRS_MIN,Eff_CRS_MAX,Eff_VRS_MAX
0,BB,0.9896,0.9962,1.0108,1.0044
1,Ita,0.9932,0.9963,1.0069,1.0039
2,Santander,0.8948,0.9254,1.1261,1.1042


In [14]:
# Salvando em excel
df_resultado.to_excel('resultado_dea.xlsx', index=False)
# Salvando a média em excel
df_media.to_excel('media_dea.xlsx', index=False)
